# Preprocessing

In [ ]:
parser.add_argument("--config_path", type=str, required=True,
                    help="TTS config file path to define audio processin parameters.")
parser.add_argument("--out_path", default=None, type=str,
                    help="directory to save the output file.")


In [17]:
!python /home/dias/TTS/TTS/TTS/bin/compute_statistics.py --config_path /home/dias/TTS/TTS/TTS/tts/configs/config.json --out_path /home/dias/Downloads/Ruslan_mono/

 > Setting up Audio Processor...
 | > sample_rate:16000
 | > num_mels:80
 | > min_level_db:-100
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:20
 | > fft_size:1024
 | > power:1.5
 | > preemphasis:0.0
 | > griffin_lim_iters:60
 | > signal_norm:False
 | > symmetric_norm:True
 | > mel_fmin:50.0
 | > mel_fmax:7600.0
 | > spec_gain:1.0
 | > stft_pad_mode:reflect
 | > max_norm:4.0
 | > clip_norm:True
 | > do_trim_silence:True
 | > trim_db:60
 | > do_sound_norm:False
 | > stats_path:None
 | > hop_length:256
 | > win_length:1024
 | > Found 22200 files in /home/dias/Downloads/Ruslan_mono
 > There are 21978 files.
100%|█████████████████████████████████████| 21978/21978 [06:13<00:00, 58.88it/s]
 > Avg mel spec mean: -2.290687005268809
 > Avg mel spec scale: 0.8254628181086708
 > Avg linear spec mean: -1.633475909923078
 > Avg lienar spec scale: 0.8549587686344885
 > scale_stats.npy is saved to /home/dias/Downloads/Ruslan_mono/scale_stats.npy


# Training encoder

In [ ]:
#logs are hidden as Github could not open the page
!python /home/dias/TTS/TTS/TTS/bin/train_tts.py --config_path /home/dias/TTS/TTS/TTS/tts/configs/config.json --continue_path /home/dias/Downloads/Models/Ruslan/ljspeech-ddc-November-09-2020_11+57AM-0000000

# Training vocoder

In [1]:
cd /home/dias/TTS/TTS/TTS/

/home/dias/TTS/TTS/TTS


In [ ]:
#logs are hidden as Github could not open the page
!CUDA_VISIBLE_DEVICES='0' python bin/train_vocoder.py --config_path vocoder/configs/multiband_melgan_config.json --continue_path /home/dias/Downloads/Models/Ruslan/multiband-melgan-November-10-2020_08+55AM-af6f862

# Testing results

In [3]:
def tts(model, text, CONFIG, use_cuda, ap, use_gl, figures=True):
    t_1 = time.time()
    waveform, alignment, mel_spec, mel_postnet_spec, stop_tokens, inputs = synthesis(model, text, CONFIG, use_cuda, ap, speaker_id, style_wav=None,
                                                                             truncated=False, enable_eos_bos_chars=CONFIG.enable_eos_bos_chars)
    # mel_postnet_spec = ap._denormalize(mel_postnet_spec.T)
    if not use_gl:
        waveform = vocoder_model.inference(torch.FloatTensor(mel_postnet_spec.T).unsqueeze(0))
        waveform = waveform.flatten()
    if use_cuda:
        waveform = waveform.cpu()
    waveform = waveform.numpy()
    rtf = (time.time() - t_1) / (len(waveform) / ap.sample_rate)
    tps = (time.time() - t_1) / len(waveform)
    print(waveform.shape)
    print(" > Run-time: {}".format(time.time() - t_1))
    print(" > Real-time factor: {}".format(rtf))
    print(" > Time per step: {}".format(tps))
    IPython.display.display(IPython.display.Audio(waveform, rate=CONFIG.audio['sample_rate']))  
    return alignment, mel_postnet_spec, stop_tokens, waveform

In [4]:
import sys
import os
import torch
import time
import IPython

# for some reason TTS installation does not work on Colab
sys.path.append('TTS_repo')

from TTS.utils.io import load_config
from TTS.utils.audio import AudioProcessor
from TTS.tts.utils.generic_utils import setup_model
from TTS.tts.utils.text.symbols import symbols, phonemes
from TTS.tts.utils.synthesis import synthesis
from TTS.tts.utils.io import load_checkpoint

In [11]:
# model paths
path_vocoder ='/home/dias/Downloads/Models/Ruslan/multiband-melgan-November-10-2020_08+55AM-af6f862'
path_tts = '/home/dias/Downloads/Models/Ruslan/ljspeech-ddc-November-09-2020_11+57AM-0000000'
TTS_MODEL = path_tts+"/best_model.pth.tar"
TTS_CONFIG = path_tts+"/config.json"
VOCODER_MODEL = path_vocoder+"/best_model.pth.tar"
VOCODER_CONFIG = path_vocoder+"/config.json"


In [12]:
# load configs
TTS_CONFIG = load_config(TTS_CONFIG)
VOCODER_CONFIG = load_config(VOCODER_CONFIG)

In [13]:
# load the audio processor
ap = AudioProcessor(**TTS_CONFIG.audio)         

 > Setting up Audio Processor...
 | > sample_rate:16000
 | > num_mels:80
 | > min_level_db:-100
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:20
 | > fft_size:1024
 | > power:1.5
 | > preemphasis:0.0
 | > griffin_lim_iters:60
 | > signal_norm:True
 | > symmetric_norm:True
 | > mel_fmin:50.0
 | > mel_fmax:7600.0
 | > spec_gain:1.0
 | > stft_pad_mode:reflect
 | > max_norm:4.0
 | > clip_norm:True
 | > do_trim_silence:True
 | > trim_db:60
 | > do_sound_norm:False
 | > stats_path:/home/dias/Downloads/Ruslan_mono/scale_stats.npy
 | > hop_length:256
 | > win_length:1024


In [14]:
# LOAD TTS MODEL
# multi speaker 
speaker_id = None
speakers = []
# load the model
# runtime settings
use_cuda = True
num_chars = len(phonemes) if TTS_CONFIG.use_phonemes else len(symbols)
model = setup_model(num_chars, len(speakers), TTS_CONFIG)

# load model state
cp =  torch.load(TTS_MODEL, map_location=torch.device('cpu'))

# load the model
model.load_state_dict(cp['model'])
if use_cuda:
    model.cuda()
model.eval()

# set model stepsize
if 'r' in cp:
    model.decoder.set_r(cp['r'])

 > Using model: Tacotron2


In [15]:


from TTS.vocoder.utils.generic_utils import setup_generator

# LOAD VOCODER MODEL
vocoder_model = setup_generator(VOCODER_CONFIG)
vocoder_model.load_state_dict(torch.load(VOCODER_MODEL, map_location="cpu")["model"])
vocoder_model.remove_weight_norm()
vocoder_model.inference_padding = 0

ap_vocoder = AudioProcessor(**VOCODER_CONFIG['audio'])    
if use_cuda:
    vocoder_model.cuda()
vocoder_model.eval()



 > Generator Model: multiband_melgan_generator
 > Setting up Audio Processor...
 | > sample_rate:16000
 | > num_mels:80
 | > min_level_db:-100
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:0
 | > fft_size:1024
 | > power:None
 | > preemphasis:0.0
 | > griffin_lim_iters:None
 | > signal_norm:True
 | > symmetric_norm:True
 | > mel_fmin:50.0
 | > mel_fmax:7600.0
 | > spec_gain:1.0
 | > stft_pad_mode:reflect
 | > max_norm:4.0
 | > clip_norm:True
 | > do_trim_silence:True
 | > trim_db:60
 | > do_sound_norm:False
 | > stats_path:/home/dias/Downloads/Ruslan_mono/scale_stats.npy
 | > hop_length:256
 | > win_length:1024


MultibandMelganGenerator(
  (layers): Sequential(
    (0): ReflectionPad1d((3, 3))
    (1): Conv1d(80, 384, kernel_size=(7,), stride=(1,))
    (2): LeakyReLU(negative_slope=0.2)
    (3): ConvTranspose1d(384, 192, kernel_size=(16,), stride=(8,), padding=(4,))
    (4): ResidualStack(
      (blocks): ModuleList(
        (0): Sequential(
          (0): LeakyReLU(negative_slope=0.2)
          (1): ReflectionPad1d((1, 1))
          (2): Conv1d(192, 192, kernel_size=(3,), stride=(1,))
          (3): LeakyReLU(negative_slope=0.2)
          (4): Conv1d(192, 192, kernel_size=(1,), stride=(1,))
        )
        (1): Sequential(
          (0): LeakyReLU(negative_slope=0.2)
          (1): ReflectionPad1d((3, 3))
          (2): Conv1d(192, 192, kernel_size=(3,), stride=(1,), dilation=(3,))
          (3): LeakyReLU(negative_slope=0.2)
          (4): Conv1d(192, 192, kernel_size=(1,), stride=(1,))
        )
        (2): Sequential(
          (0): LeakyReLU(negative_slope=0.2)
          (1): Reflectio

In [31]:
sentence =  "Мой папа много работает"
align, spec, stop_tokens, wav = tts(model, sentence, TTS_CONFIG, use_cuda, ap, use_gl=False, figures=True)

(23552,)
 > Run-time: 0.07555699348449707
 > Real-time factor: 0.05127991671147554
 > Time per step: 3.2050656559674637e-06
